### Imports

In [1]:
#Time
import time

#Random
import random

#CSV
import csv  

#OS
import os

# Selenium
# from selenium import webdriver
# from selenium.webdriver.firefox.service import Service
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.by import By
# from selenium.common.exceptions import TimeoutException
# from selenium.common.exceptions import WebDriverException
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import trafilatura
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import glob
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import reuters
import numpy as np
import pandas as pd
from collections import Counter
from urllib.parse import urljoin, urlparse, urlunparse
import json


### Reddit Scrapper

init -> (userAgent)(no need)
→ (login)(Optional)
→ loop { 
     rateLimiter
     randomDelay
     fetch/searchSubReddit
     cleanData(deduplication + canonicalization)
     saveData(incremental)
  }

In [ ]:
#init
def init():
    try:
        service = Service("/usr/bin/geckodriver")
        driver = webdriver.Firefox(service=service)
        return driver
    except WebDriverException as e:
        print("WebDriver error happened:", e)
        return None
    finally:
        pass

In [ ]:
#randomDelay
def randomDelay():
    time.sleep(random.uniform(2,5))

In [ ]:
#rateLimiter
def rate_limiter(searchSubReddit,totalPost, posts_before_cooldown=30, cooldown_time=60):
    counter = 0
    for _ in range(totalPost):
        searchSubReddit()
        counter += 1

        randomDelay()

        if counter >= posts_before_cooldown:
            print("Cooldown Between 30 Topics")
            time.sleep(cooldown_time)
            counter = 0

In [ ]:
#subReddits List
subRedditsNews = [
    "https://www.reddit.com/r/news/",
    "https://www.reddit.com/r/worldnews/",
    "https://www.reddit.com/r/realbbcnews/",
    "https://www.reddit.com/r/world24x7hr/"
] 

In [ ]:
#fetch/searchSubReddit
def searchRedditNews(subRedditsNews):
    driver = init()
    for item in range(len(subRedditsNews)):        
        driver.get(subRedditsNews[item])
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[-1])
        
        randomDelay()
    driver.close()
    driver.switch_to.window(driver.window_handles[0])


In [ ]:
# searchRedditNews(subRedditsNews)

### Web Crawler

In [2]:
news_seeds = [
    "https://www.theguardian.com", # 54
    "https://www.aljazeera.com/news/", # 19
    "https://www.nytimes.com/section/world",# forbidden url
    "https://www.nbcnews.com/world",# keyword: world , 22
]
visited = set()
articles = []

In [ ]:
visited = set()
articles = []

def canonicalize(url):
    """Remove fragments and query params from URL."""
    parsed = urlparse(url)
    # (scheme, netloc, path, params, query, fragment)
    return urlunparse((parsed.scheme, parsed.netloc, parsed.path, "", "", ""))


def extract_text(html, url):
    """Try to extract main article text using trafilatura, fallback to <p> tags."""
    text = trafilatura.extract(html, url=url)
    if text:
        return text
    else:
        soup = BeautifulSoup(html, "html.parser")
        return " ".join([p.get_text() for p in soup.find_all("p")])

def crawl(url, index, depth=1):
    url = canonicalize(url)

    if depth == 0 or url in visited:
        return
    
    if True:
        print(f"Crawling: {url}")
        visited.add(url)
        
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
        except Exception as e:
            print(f"Failed to fetch {url}: {e}")
            return
        
        html = response.text
        
        text = extract_text(html, url)
        
        if text:
            if re.search(r"/20\d{2}/", url):  
                articles.append({"url": url, "id": 0, "title": 0, "writer": 0,  "raw_text": text})
                print(f"Saved article ({len(text)} chars)")
        else:
            print(f"Not an article, but checking links...")
        
        # Parse links
        soup = BeautifulSoup(html, "html.parser")
        for link in soup.find_all("a", href=True):
            new_url = urljoin(url, link["href"])
            
            # Filter: must be within same domain as seed
            if new_url not in visited and url.split("/")[2] in new_url:
                crawl(new_url, index, depth - 1)


crawl(news_seeds[0], 0, depth=3)

print(f"Collected {len(articles)} articles")

In [ ]:
print(articles[310]['raw_text'])

In [ ]:
with open("articles.json", "w", encoding="utf-8") as f:
    json.dump(articles, f, ensure_ascii=False, indent=2)

In [3]:
articles = []
with open("articles.json", "r", encoding="utf-8") as f:
    articles = json.load(f)

print(f"Restored {len(articles)} articles")
print(articles[0]["raw_text"])  # check 

Restored 537 articles
When a history of resistance to the lurching authoritarianism of Donald Trump’s second presidency is written, it could well begin on 11 April 2025, inside a small immigration courtroom in remote, central Louisiana.
It was there, in the early afternoon, that a slight young man dressed in a blue uniform jumpsuit spoke calmly but directly to the new administration – away from the gaze of television cameras and 1,000 miles (1,610km) from his friends and family. Mahmoud Khalil, the Columbia University graduate and Palestinian organiser, had been arrested a month earlier – snatched from the lobby of his Manhattan apartment building as he returned home with his wife. Now, detained in the small town of Jena, he sat before a judge who had just ruled that he was eligible to be deported from the United States purely for his political views.
Khalil asked for permission to speak. He paused for a moment, before sharply rebuking the jurist who continued to hold his fate in her h

### CSV

In [ ]:
def saveCsv(
    type,
    source,
    url,
    id,
    topic,
    author,
    timeRelease,
    vote,
    commentsCount,
    language,
    token,
    predicte,
):
    columnTopics = [
        "Type",
        "Source",
        "URL",
        "ID",
        "Topic",
        "Author",
        "TimeRelease",
        "Vote",
        "CommentsCount",
        "Language",
        "Token",
        "Predicte",
    ]
    row = [
        type,
        source,
        url,
        id,
        topic,
        author,
        timeRelease,
        vote,
        commentsCount,
        language,
        token,
        predicte,
    ]
    
    fileName = "RecordsNewsReddit.csv"
    file_exists = os.path.exists(fileName)
    
    with open(fileName, mode="a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        
        if not file_exists:
            writer.writerow(columnTopics)
        
        writer.writerow(row)

In [ ]:
# #Example Save Data
# saveCsv(
#     type="Reddit",
#     source="r/news",
#     url="https://reddit.com/r/news/xyz",
#     id="abc123",
#     topic="Breaking News",
#     author="user42",
#     timeRelease="2025-09-08 12:00",
#     vote=420,
#     commentsCount=69,
#     language="en",
#     token="breaking,news",
#     predicte="Politics"
# )

### Preprocessing

In [ ]:
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("reuters")

In [4]:
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [5]:
def normalize(articles):
    for article in articles:
        text = article['raw_text']
        text = text.lower()
        text = text.replace("\n", " ")

        # Remove URLs
        text = re.sub(r'http\S+|www\.\S+', '', text)
        # Remove special characters (keep only words and numbers)
        text = re.sub(r'[^a-z0-9\s]', ' ', text)

        article['normalize_text'] = text

# normalize(articles)
# print(articles[10]['normalize_text'])

In [6]:
def tokenize(articles):
    for article in articles:
        tokens = nltk.word_tokenize(article['normalize_text'])
    
        # Remove stopwords
        article["tokens"] = [t for t in tokens if t not in stop_words] 

# tokenize(articles)
# print(articles[10]['tokens'])

In [7]:
def stem(articles):
    for article in articles:
        article["tokens"] = [stemmer.stem(t) for t in article["tokens"]]

# stem(articles)
# print(articles[10]['tokens'])

In [8]:
def lemmatize(articles):
    for article in articles:
        article["tokens"] = [lemmatizer.lemmatize(t) for t in article["tokens"]]

# lemmatize(articles)
# print(articles[10]['tokens'])

In [9]:
def preprocess(articles):
    normalize(articles)
    tokenize(articles)
    stem(articles)
    # lemmatize(articles)
    
preprocess(articles)

### Classification

In [10]:
DATA_DIR = "./dataset"
dataset_articles = []

for filename in os.listdir(DATA_DIR):
    if not filename.endswith(".sgm"):
        continue
    
    with open(os.path.join(DATA_DIR, filename), "r", encoding="latin-1") as f:
        data = f.read()
        soup = BeautifulSoup(data, "html.parser")
        
        for reuters in soup.find_all("reuters"):
            text = ""
            if reuters.body:
                text = reuters.body.get_text(strip=True)
            topics = [t.get_text() for t in reuters.topics.find_all("d")] if reuters.topics else []
            
            if text.strip():
                dataset_articles.append({
                    "id": reuters["newid"],
                    "title": reuters.title.get_text(strip=True) if reuters.title else None,
                    "raw_text": text,
                    "topics": topics
                })

print(dataset_articles[10]["raw_text"])
print(dataset_articles[10]["title"])

Shr 34 cts vs 1.19 dlrs
    Net 807,000 vs 2,858,000
    Assets 510.2 mln vs 479.7 mln
    Deposits 472.3 mln vs 440.3 mln
    Loans 299.2 mln vs 327.2 mln
    Note: 4th qtr not available. Year includes 1985
extraordinary gain from tax carry forward of 132,000 dlrs, or
five cts per shr.
 Reuter

COBANCO INC <CBCO> YEAR NET


In [11]:
# dataset_size = 200000

# articles = []
# for fileid in reuters.fileids()[:dataset_size]:
#     text = reuters.raw(fileid)
#     cats = reuters.categories(fileid)  # some docs have multiple categories
#     if cats:
#         articles.append({"raw_text": text, "label": cats[0]})

# articles[10]["label"]

In [12]:
preprocess(dataset_articles)

In [13]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=5000)
X_reuters = vectorizer.fit_transform([" ".join(dataset_article["tokens"]) for dataset_article in dataset_articles])
X_crawled = vectorizer.transform([" ".join(article["tokens"]) for article in articles])

In [19]:
category_vocab = {}

# Only keep articles with at least one topic
valid_articles = [art for art in dataset_articles if art["topics"]]

# collect unique labels (using the first topic as representative)
labels = list(set(art["topics"][0] for art in valid_articles))

for label in labels:
    # find indices of docs belonging to this label
    idx = [i for i, art in enumerate(valid_articles) if art["topics"][0] == label]
    
    if not idx:  # skip if no documents
        continue
    
    # average TF-IDF weights for this category
    avg_weights = np.asarray(X_reuters[idx].mean(axis=0)).ravel()
    
    # get indices of top 50 highest-weighted terms
    top_idx = avg_weights.argsort()[::-1][:50]
    
    # get actual term strings
    terms = vectorizer.get_feature_names_out()[top_idx]
    
    # get their corresponding weights
    weights = avg_weights[top_idx]
    
    # save into dictionary
    category_vocab[label] = dict(zip(terms, weights))


In [20]:
def score_article(article_vector):
    scores = {}
    feature_names = vectorizer.get_feature_names_out()
    vec_array = article_vector.toarray().ravel()
    for label, vocab in category_vocab.items():
        score = sum(vec_array[feature_names.tolist().index(term)] * w
                    for term, w in vocab.items() if term in feature_names)
        scores[label] = score
    return scores

for article, x in zip(articles, X_crawled):
    scores = score_article(x)  # returns a dict of {label: score}
    article["scores"] = scores
    article["predicted"] = max(scores, key=scores.get)


In [21]:
articles[10]["predicted"]
print(Counter(a["predicted"] for a in articles))



Counter({'tapioca': 150, 'barley': 108, 'hk': 108, 'naphtha': 38, 'fishmeal': 31, 'f-cattle': 18, 'cpu': 17, 'nzdlr': 16, 'soy-meal': 12, 'groundnut': 7, 'coconut': 6, 'platinum': 4, 'plywood': 4, 'inventories': 3, 'rice': 3, 'rapeseed': 2, 'soybean': 2, 'palm-oil': 2, 'income': 1, 'hog': 1, 'corn': 1, 'austdlr': 1, 'rand': 1, 'jet': 1})


In [22]:
for article in articles[:5]:
    print(article["raw_text"])
    print(f"### predicted class: {article['predicted']}")
    print("*"*20)

When a history of resistance to the lurching authoritarianism of Donald Trump’s second presidency is written, it could well begin on 11 April 2025, inside a small immigration courtroom in remote, central Louisiana.
It was there, in the early afternoon, that a slight young man dressed in a blue uniform jumpsuit spoke calmly but directly to the new administration – away from the gaze of television cameras and 1,000 miles (1,610km) from his friends and family. Mahmoud Khalil, the Columbia University graduate and Palestinian organiser, had been arrested a month earlier – snatched from the lobby of his Manhattan apartment building as he returned home with his wife. Now, detained in the small town of Jena, he sat before a judge who had just ruled that he was eligible to be deported from the United States purely for his political views.
Khalil asked for permission to speak. He paused for a moment, before sharply rebuking the jurist who continued to hold his fate in her hands by throwing her o